# **Part II: Analysis**
team member: Shaoying Zheng, Zhongrui Ning, Xiao Pu


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind
from wordcloud import WordCloud
import matplotlib.ticker as mticker

In [ ]:
# load data
df = pd.read_csv('Projectdata/job_company_pair.csv')
df.head()

In [ ]:
#df[df['annual_max_salary']<df['annual_med_salary']]

## Descriptive Statistics: 

Firstly, let's check the columns of our data

In [ ]:
df.columns

In those columns, `job_id` is the unique value for each job
For the purpose of our analysis, which aiming at job hunting recommendation, we would focus on those following columns: 
- Salary of the job: described by `annual_min_salary`, `annual_med_salary` and `annual_max_salary`
- Skills the job needs: described by `skill_abr`, `skill_name` specifically and `skill_catogory` generally. 
- Size and popularity of the company：described by `employee_count`,`follower_count`, `company_size` and `employee_follower_ratio`
- Industry fields: described by `industry_name` and `industry_id`

### Salary description

`annual_min_salary`, `annual_med_salary` and `annual_max_salary` are all numerical float variables, each columns contains the salary description of the job, but not every job have those three kind of salary descriptions (minimum, median and maximum). 

In [ ]:
df[['annual_min_salary', 'annual_med_salary', 'annual_max_salary']].describe().T

- We handled the missing values in part 1, and filled all the missing values
- In Part I, we have briefly described the relationship between `annual_med_salary` and occupation type/company size. 
- We didn't use `annual_min_salary` and `annual_max_salary` before. 
- We may use `annual_med_salary` as the prediction variable. 

### Skills description

`skill_abr`, `skill_name` and `skill_category` are all text variables, contains the arbitrary of the skills, the full name of the skills and the category of the skills. 

In [ ]:
df. skill_category.value_counts().describe()

In [ ]:
df. skill_category.value_counts().head(10)

As we can see, the top three skill categories in the job market (among others) are business, management, and engineering skills, which partly reflects the popular hiring needs right now.

### Size and popularity of the company analysis
`employee_count`,`follower_count` and `company_size` are columns that describes the number of employees, size of the company and number of followers in Leaderboard, these attributes can help us understand the size of the company and its employees, where the size of the company is described by an integer from 1 to 7, with size increasing with the number

In [ ]:
df[['employee_count', 'follower_count', 'company_size']].describe().T

As can be seen from the description, nearly half of the jobs in this dataset come from larger companies, with fewer jobs available at smaller companies, while the number of employees varies considerably among similarly sized companies, which may be related to the industrial sector to which the companies belong to. 

In [ ]:
df.employee_follower_ratio.describe()

In the analysis of  `emoloyee_follower_ratio`, we can see that the number of employees/followers ratio for each company seems to be very low, which also suggests that the job market is very competitive. 

### Industry fields analysis

In [ ]:
print('The 10 industrial sectors offering the most jobs', df.industry_name.value_counts().head(10), '\n')
print('The 10 industrial sectors offering the least jobs', df.industry_name.value_counts().tail(10), '\n')

From the analysis we can see that industries such as healthcare and human resources are providing the most jobs, while the industrial sectors that are nearing job saturation are security systems, computer networks, etc., but we can see that these sectors overlap in some ways.

## Inferential Statistics

We've briefly seen the relationship between salary and skill categories .Let's take a deeper look into the salary by different skill categories.

In [ ]:
print(df.groupby('skill_category')['annual_med_salary'].describe())

We first filter those extreme values before doing tests to the data.

In [ ]:
df_filtered = df[(df['annual_med_salary'] >= 7500) & (df['annual_med_salary'] <= 1000000)]

Since the distribution of salaries is skewed rather than simply normal, we chose to use the Kruskal-Wallis H Test. 

We want to check if median salaries have any statistically significant difference between different skill categories to help job seekers to enrich their skills to be more competitive in the workforce and to choose a career in the future based on their own circumstances, preferences, and salary level.

**H0: The distribution of median salaries across the different job categories is the same.**

**H1: There is at least one job category where the distribution of median salaries is different from the others.**

### [Kruskal-Wallis H Test](https://en.wikipedia.org/wiki/Kruskal%E2%80%93Wallis_test)


The formula for the Kruskal-Wallis H test is as follows:

$$
H = \frac{12}{N(N+1)} \sum_{i=1}^{k} \frac{R_i^2}{n_i} - 3(N+1)
$$

where:

- \( N \) is the total sample size.
- \( k \) is the number of categories.
- \( R_i \) is the rank sum for each group.
- \( n_i \) is the number of samples in each group.



In [ ]:
from scipy.stats import kruskal

# Assuming `df` is your DataFrame and `salary` is the column with median salary
categories = df_filtered['skill_category'].unique()
salaries = [df_filtered[df_filtered['skill_category'] == cat]['annual_med_salary'] for cat in categories]

# Perform Kruskal-Wallis test
stat, p_value = kruskal(*salaries)

print("Kruskal-Wallis H-statistic:", stat)
print("p-value:", p_value)

# Interpretation
if p_value < 0.05:
    print("There is a statistically significant difference in median salaries between job categories.")
else:
    print("There is no statistically significant difference in median salaries between job categories.")


Based on the Kruskal-Wallis H test, we reject the null hypothesis at the 0.05 significance level. This indicates that there is a statistically significant difference in the distribution of median salaries across different job categories.

Let's then add labels to categorize the salary and do some tests.

In [ ]:
def categorize_salary(group):
    low_salary_threshold = group['annual_med_salary'].quantile(0.25)
    high_salary_threshold = group['annual_med_salary'].quantile(0.75)
    if (low_salary_threshold==0):
        print('found')
    #print(f"Low Salary Threshold: {low_salary_threshold}, High Salary Threshold: {high_salary_threshold}")
    if low_salary_threshold == high_salary_threshold: 
        if group['annual_med_salary'].min() == group['annual_med_salary'].max():    
            group['salary_level'] = 'Medium Salary'
            return group
        else:     
            low_salary_threshold = group['annual_med_salary'].min()
            high_salary_threshold = group['annual_med_salary'].max()
    group['salary_level'] = pd.cut(group['annual_med_salary'], 
                                   bins=[0, low_salary_threshold, high_salary_threshold, float('inf')], 
                                   labels=['Low Salary', 'Medium Salary', 'High Salary'])
    return group
df_labeled = df.groupby('industry_name').apply(categorize_salary).reset_index(drop=True)

#print(df[['industry_name', 'annual_med_salary', 'salary_level']].head())


In [ ]:
df_labeled['industry_name'].value_counts().head(10)

In [ ]:
import pandas as pd
from scipy.stats import chi2_contingency


contingency_table = pd.crosstab(df_labeled['skill_category'], df_labeled['salary_level'])
print(contingency_table)

#chi2 test
chi2, p_value, dof, expected = chi2_contingency(contingency_table)

print("Chi-square statistic:", chi2)
print("p-value:", p_value)
print("Degrees of freedom:", dof)


if p_value < 0.05:
    print("There are significant differences in salary levels across skill categories.")
else:
    print("There are no significant differences in salary levels across skill categories.")





In [ ]:
observed = contingency_table.values
expected = pd.DataFrame(expected, index=contingency_table.index, columns=contingency_table.columns)

# calculate deviation
deviation = observed - expected


print("Observed:\n", contingency_table)
print("\nExpected:\n", expected)
print("\nDeviation:\n", deviation)



# calculate std_residuals
standardized_residuals = (observed - expected) / np.sqrt(expected)

print("\nStandardized Residuals:\n", standardized_residuals)


plt.figure(figsize=(10, 8))
sns.heatmap(standardized_residuals, annot=True, cmap="coolwarm", center=0)
plt.title("Standardized Residuals Heatmap")
plt.show()


We see that Technical & Engineering and Legal & Compliance skills tend to relate more with High Salary.
Business and Administration skills are predominantly in the Low Salary category.


## Graphical Analysis

In [ ]:
text = ' '.join(df['skill_name'].dropna())
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

We first use word cloud to have a general look at the most common skills accross all industries. It seems that Information technology and sales are the hottest. Let's check this in a more readable way.

In [ ]:
skill_counts = df['skill_name'].value_counts().reset_index()
skill_counts.columns = ['skill_name', 'count']

In [ ]:
plt.figure(figsize=(12, 6))
sns.barplot(data=skill_counts.head(20), x='count', y='skill_name')
plt.title("Top 20 Most Common Skills Across All Industries")
plt.xlabel("Frequency")
plt.ylabel("Skill Name")
plt.show()

Let's then check the heatmap to see how job requirements vary across different industries.

In [ ]:
industry_skill_counts = df.groupby(['industry_name', 'skill_name']).size().unstack(fill_value=0)
plt.figure(figsize=(20, 15))
sns.heatmap(
    industry_skill_counts, 
    cmap="viridis",  
    cbar_kws={'label': 'Skill Frequency'}  
)


plt.title("Skill Frequency by Industry", fontsize=18)
plt.xlabel("Skills", fontsize=14)
plt.ylabel("Industry", fontsize=14)


plt.xticks(rotation=45, ha='right', fontsize=10)
plt.yticks(fontsize=10)
plt.show()

It's unreadable. So we filter the data.

In [ ]:
top_industries = industry_skill_counts.sum(axis=1).nlargest(20).index
top_skills = industry_skill_counts.sum(axis=0).nlargest(20).index


filtered_data = industry_skill_counts.loc[top_industries, top_skills]

plt.figure(figsize=(18, 12))
sns.heatmap(
    filtered_data, 
    cmap="viridis", 
    linewidths=0,  
    cbar_kws={'label': 'Skill Frequency'}
)


plt.title("Top Skill Frequency by Industry", fontsize=20)
plt.xlabel("Skills", fontsize=15)
plt.ylabel("Industry", fontsize=15)
plt.xticks(rotation=45, ha='right', fontsize=12)
plt.yticks(fontsize=12)
plt.show()

We see that skills such as Information Technology, Sales, and Management have high demand across multiple industries, reflecting their importance and versatility in the modern workplace. There are also industry-Specific Skills like healthcare skills in the medical field, engineering skills in manufacturing and do not hold a dominant position in cross-industry comparisons.

In [ ]:
# First, let's check difference of scale and influence(i.e., number of employee, number of follower and employee-follower ratio) for companies from different industries
# Compute industries with most industries
scale= df.groupby('industry_name')[['employee_count', 'follower_count']].sum()
scale.reset_index(inplace=True)
top_10_scale = scale.sort_values('employee_count',ascending=False).head(10)
top_10_scale

We can see top 10 industries with the most job postings. We can see that industries with the most job postings are IT, Finance, and Healthcare.

In [ ]:
top_10_influence = scale.sort_values('follower_count',ascending=False).head(10)
top_10_influence

We can see top 10 industries with the most followers. We can see that industries with the most followers are similar to industries with the most job postings, while adding food and beverage services in.

In [ ]:
# For different industries within top 10 scale and influence, check the distribution of employee_count, follower_count and employee-follower ratio
top_10_industry = list(set(top_10_scale['industry_name']).union(set(top_10_influence['industry_name'])))
top_10_industry_df = df[df['industry_name'].isin(top_10_industry)]
top_10_industry_df = top_10_industry_df[['industry_name', 'employee_count', 'follower_count', 'employee_follower_ratio']]
# create subsets for each industries and draw boxplot
fig, ax = plt.subplots(3, 1, figsize=(15, 15))
for i, col in enumerate(['employee_count', 'follower_count', 'employee_follower_ratio']):
    sns.boxplot(x='industry_name', y=col, data=top_10_industry_df, ax=ax[i],hue = 'industry_name')
    ax[i].set_title(col)
    ax[i].set_xticklabels(ax[i].get_xticklabels(), rotation=45, ha='right', rotation_mode="anchor")
ax[1].yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'{int(x):,}'))
plt.tight_layout()
plt.show()

We can see that software development and Information services industries have the highest overall employee count overall. Which should be the most popular industries that college students should focus on.

## Compartive Analysis

For this part, we will focus on company size's impact on skill requirements.

In [ ]:
# Group by company size
company_size_skill = df_labeled.groupby(['company_size','skill_category']).size().unstack()
company_size_skill= company_size_skill.div(company_size_skill.sum(axis=1), axis=0) * 100
company_size_skill = company_size_skill.fillna(0)
company_size_skill.reset_index(inplace=True)
# company_size_skill = company_size_skill.melt(id_vars='company_size', var_name='skill_category', value_name='percentage')
company_size_skill

In [ ]:
# Draw stacked barplot
plt.figure(figsize=(20, 8))
company_size_skill.plot(kind='bar', x='company_size',stacked=True, colormap="tab20", figsize=(15, 8))
plt.title('Skill Category Distribution by Company Size')
plt.xlabel('Company Size')
plt.ylabel('Percentage')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Skill Category')
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
plt.tight_layout()
plt.show()

We can see that the distribution of skill categories is similar across different company sizes. 

Then, let's check difference of scale and influence(i.e., number of employee, number of follower and employee-follower ratio) for companies from different industries。

In [ ]:
# First, let's check difference of scale and influence(i.e., number of employee, number of follower and employee-follower ratio) for companies from different industries
# Compute industries with most industries
scale= df.groupby('industry_name')[['employee_count', 'follower_count']].sum()
scale.reset_index(inplace=True)
top_10_scale = scale.sort_values('employee_count',ascending=False).head(10)
top_10_scale

We can see top 10 industries with the most job postings. We can see that industries with the most job postings are IT, Finance, and Healthcare.

In [ ]:
top_10_influence = scale.sort_values('follower_count',ascending=False).head(10)
top_10_influence

We can see top 10 industries with the most followers. We can see that industries with the most followers are similar to industries with the most job postings, while adding food and beverage services in.

In [ ]:
# For different industries within top 10 scale and influence, check the distribution of employee_count, follower_count and employee-follower ratio
top_10_industry = list(set(top_10_scale['industry_name']).union(set(top_10_influence['industry_name'])))
top_10_industry_df = df[df['industry_name'].isin(top_10_industry)]
top_10_industry_df = top_10_industry_df[['industry_name', 'employee_count', 'follower_count', 'employee_follower_ratio']]
# create subsets for each industries and draw boxplot
fig, ax = plt.subplots(3, 1, figsize=(15, 15))
for i, col in enumerate(['employee_count', 'follower_count', 'employee_follower_ratio']):
    sns.boxplot(x='industry_name', y=col, data=top_10_industry_df, ax=ax[i],hue = 'industry_name')
    ax[i].set_title(col)
    ax[i].set_xticklabels(ax[i].get_xticklabels(), rotation=45, ha='right', rotation_mode="anchor")
ax[1].yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'{int(x):,}'))
plt.tight_layout()
plt.show()

## Multivariate Analysis

In [ ]:
# We want to perform analysis of industry, skill category and salary level, and visualize by cross-tabulation
industry_skill_salary = pd.crosstab(
    index=[df_labeled['industry_name'], df_labeled['skill_category']], 
    columns=df_labeled['salary_level'], 
    margins=True,
    margins_name='Total'
)
industry_skill_salary.reset_index(inplace=True)
industry_skill_salary

In [ ]:
# Sort by high salary level
industry_skill_salary.sort_values('High Salary', ascending=False, inplace=False).head(5)

For all job postings across every industry that offered a high salary, the skill_category is listed as "other", which means the job may requires a variety of skills and maybe not suitable for a new graduate job seeker.

In [ ]:
# Eliminate skill_category list as 'Other'
industry_skill_salary[industry_skill_salary['skill_category'] != 'Other'].sort_values('High Salary', ascending=False)[1:6]

Here we can see Technical & Engineering, Finance, and Business are most highly paid skills.

## Synthesis

From our analysis, we were able to get some insight into our original research questions:
- What are the most common skills listed in job postings across various industries?

- How do job requirements vary across different industries?

- How could job hunters with different background find suitable jobs?
  
  1. In our **descriptive statistics** part,we analyzed salaries for these occupations，chose median salary as our basis of salary analysis, identified common skill requirements across industries (e.g., information technology and data analytics skills are valued across multiple fields), and conducted a preliminary analysis of the types and fundamentals of employer industry sectors across the job market, as well as the potential intensity of competition in the job market. 
  2. In our **Inferential statistics** part, we used Kruskal-Wallis H Test to analyze if there is a statistically significant difference in median salary between different skill categories. The hypothesis test was validated, indicating that different skill categories do affect salaries, with industries associated with Technical & Engineering and Legal & Compliance skills having relatively higher salaries, while Business and Administration skills possess significantly less salary level. 
  3. The **graphical statistics** part：We created 3 types of charts to describe and analyze our dataset in a comprehensive way: 1.Using wordcloud map to get the keywords in skill categories and industry names. 2. Using bar charts to count the 20 hottest skills in the job market. 3. Using heat maps to count the correlation between the hottest skills and the industrial sector that the job belongs to, to find the hottest pairings.
  4. In our **comparative statistics**, we 1. compare the types of skills required for jobs in companies of different sizes, revealing more visually the differences in job skill preferences between small and large companies. 2. created boxplots with the number of employees, the number of followers, and the ratio of employees/followers of the firms belonging to the different industrial sectors We created box plots of the number of employees, followers, and employee/follower ratio of companies belonging to different industrial sectors to more visually analyze which companies are receiving more attention in the job market.
  5. We used industry, salary level and skill categories in our **multivariable statistics**, looking for correlations between salary, type of skill and the type of industry the company belongs to, our results indicate that many jobs having a high salary needs multiple skills that may not suitable for new job hunters, and otherwise, the best-paying skill types are Technical & Engineering, Finance, and Business.

## Reflection

  Our dataset is big including almost all the job posted on linkedin from 2023 to 2024. Despite the challenges we encountered in part 1 when cleaning and merging the dataset, we still faced some new problems in this part. For example, the multiple industry fields in this huge dataset makes it hard to visualize. It was difficult to validate individually and filter them out. Also how to choose the variables and to make our analysis work as well as it should is also a difficult process. But finally we solved those problems. 
  
  The limitations of our analysis come from the presence of implausible data; due to the many missing measurements in the original dataset, we had to fill them in using our methods in Part1, which somewhat affects the validity of our numerical analyses; at the same time, due to the huge number of industry types and skill types, our classification methods/filtering methods may not be able to analyze all variables and potential combinations of analyzed, our report strives to cover the most influential and representative segments of the job market. 
  
  For further research and improvement, we may consider utilizing more powerful data processing methods to obtain more comprehensive results.
  1. Use techniques to identify outliers and potential unreasonable/wrong data. 
  2. Processing and categorizing our lists of industry types and skills more accurately.
  3. Which variables are appropriate for predictive analytics and recommendations. 